In [1]:
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

### Files Behaviours 

In [2]:
files_behaviours_dir = 'C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataEnrichment\\Step2\\Behaviours.csv'

In [3]:
df_behaviours = pd.read_csv(files_behaviours_dir, low_memory = False)
df_behaviours = df_behaviours.drop(columns=['Unnamed: 0'])
df_behaviours

,sha1,sha256,md5,action_name,action_description,maliciousness_level,area
0,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes
1,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug setunhandledexceptionfilter,setunhandledexceptionfilter detected (possible anti-debug),neutral,System Processes
2,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,cape detected threat,cape detected the asyncrat malware,suspicious,Network Activity
3,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,cmdline terminate,executed a command line with /c or /r argument to terminate command shell on completion which can be used to hide execution,neutral,System Processes
4,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,dead connect,attempts to connect to a dead ip:port (1 unique times),neutral,Network Communications
...,...,...,...,...,...,...,...
44962,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,antisandbox sleep,a process attempted to delay the analysis task.,neutral,System Processes
44963,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,dead host,connects to an ip address that is no longer responding to requests (legitimate services will remain up-and-running usually),malicious,Network Communications
44964,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,network icmp,generates some icmp traffic,suspicious,Network Activity
44965,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,networkdyndns checkip,connects to a dynamic dns domain,neutral,Network Communications


In [4]:
df_behaviours['action_name'] = df_behaviours['action_name'].fillna(' ')
df_behaviours['action_description'] = df_behaviours['action_description'].fillna(' ')
df_behaviours

,sha1,sha256,md5,action_name,action_description,maliciousness_level,area
0,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes
1,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug setunhandledexceptionfilter,setunhandledexceptionfilter detected (possible anti-debug),neutral,System Processes
2,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,cape detected threat,cape detected the asyncrat malware,suspicious,Network Activity
3,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,cmdline terminate,executed a command line with /c or /r argument to terminate command shell on completion which can be used to hide execution,neutral,System Processes
4,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,dead connect,attempts to connect to a dead ip:port (1 unique times),neutral,Network Communications
...,...,...,...,...,...,...,...
44962,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,antisandbox sleep,a process attempted to delay the analysis task.,neutral,System Processes
44963,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,dead host,connects to an ip address that is no longer responding to requests (legitimate services will remain up-and-running usually),malicious,Network Communications
44964,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,network icmp,generates some icmp traffic,suspicious,Network Activity
44965,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247391f7e009dd4e4af2d,096492414fe97b8cc2740ef65e909f54,networkdyndns checkip,connects to a dynamic dns domain,neutral,Network Communications


### NLP sul Behaviours in ogni file

#### Example of NLP

In [5]:
!python -m spacy download it_core_news_sm


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 220.2 kB/s eta 0:01:00
     --------------------------------------- 0.1/13.0 MB 328.2 kB/s eta 0:00:40
     --------------------------------------- 0.1/13.0 MB 655.8 kB/s eta 0:00:20
      -------------------------------------- 0.2/13.0 MB 695.5 kB/s eta 0:00:19
      -------------------------------------- 0.3/13.0 MB 830.3 kB/s eta 0:00:16
     - -------------------------------------- 0.4/13.0 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/13.0 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/13.0 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/13.0 MB 1.6 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/13.0 MB 1.6 MB/s e

In [6]:
import spacy

nlp = spacy.load("it_core_news_sm")

frase = "Il gatto mangia il pesce."

doc = nlp(frase)

# Estrai sostantivi e verbi dalla frase
parole_di_interesse = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]

# Stampa i risultati
print("Parole di interesse:", parole_di_interesse)

Parole di interesse: ['gatto', 'mangia', 'pesce']


#### Get Words from descriptions

In [7]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [8]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 320.0 kB/s eta 0:00:40
     --------------------------------------- 0.0/12.8 MB 325.1 kB/s eta 0:00:40
     --------------------------------------- 0.0/12.8 MB 326.8 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.8 MB 717.5 kB/s eta 0:00:18
      -------------------------------------- 0.2/12.8 MB 908.0 kB/s eta 0:00:14
      -------------------------------------- 0.2/12.8 MB 888.4 kB/s eta 0:00:15
      -------------------------------------- 0.3/12.8 MB 980.4 kB/s eta 0:00:13
     - -------------------------------------- 0.4/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.4/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.6/12.8 MB 1.2 MB/s eta 0:00:11
     -- ------------------------------------- 0.7/12

In [9]:
from tqdm import tqdm

def create_words_df(df):
    nlp = spacy.load("en_core_web_sm")
    combined_texts = df['action_name'] + '. ' + df['action_description']
    
    data_list = []
    
    for i, text in tqdm(enumerate(combined_texts), total=len(combined_texts), desc="Processing documents"):
        doc = nlp(text)
        words_interesting = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]
        
        for word in words_interesting:
            word_type = 'noun' if nlp(word)[0].pos_ == 'NOUN' else 'verb'
            data_list.append({'word': word, 'type': word_type, 'index': i})
            
    dataframe = pd.DataFrame(data_list).reset_index()
    
    return dataframe


In [12]:
words_df = create_words_df(df_behaviours)

Processing documents: 100%|██████████| 44967/44967 [48:24<00:00, 15.48it/s]  


In [13]:
words_df

,level_0,word,type,index
0,0,guardpages,noun,0
1,1,guard,verb,0
2,2,pages,noun,0
3,3,use,verb,0
4,4,detected,verb,0
...,...,...,...,...
285941,285941,features,noun,44966
285942,285942,contains,verb,44966
285943,285943,section,noun,44966
285944,285944,names,noun,44966


In [14]:
words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\ER_Schema\\Words_Behaviours.csv')

In [15]:
merged_words_df = pd.merge(df_behaviours, words_df, left_index=True, right_on='index')
merged_words_df.head()


,sha1,sha256,md5,action_name,action_description,maliciousness_level,area,level_0,word,type,index
0,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes,0,guardpages,noun,0
1,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes,1,guard,verb,0
2,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes,2,pages,noun,0
3,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes,3,use,verb,0
4,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b613587780765c8c6cd6ef1a42,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debugging.,neutral,System Processes,4,detected,verb,0


In [16]:
merged_words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\ER_Schema\\Words_Behaviours.csv')